### This file contains functions related to loading, cleaning, and preproessing data.


The link to where the data was dowloanded from : https://www.cryptodatadownload.com/data/poloniex/#google_vignette

In [ ]:
# This code loads the data from a path

import pandas as pd

def load_data(csv_path):
    # If the first row of your CSV is just a note (e.g. “Data provided by...”),
    # then skip it with skiprows=1. If not, set skiprows=0 or remove that parameter.
    
    df = pd.read_csv(
        csv_path,
        skiprows=1,            # Adjust if your first row is already column headers
        parse_dates=['date'],  # Tells Pandas to convert the 'date' column to datetime64 objects.
        infer_datetime_format=True  #Lets Pandas guess the date format more efficiently, which can speed up parsing if your file is big.

    )
    
    #Sets the "date" column as the index of the DataFrame.
    df.set_index('date', inplace=True)
    
    #sorts the DataFrame by the date/time index, just in case the rows were out of order.
    df.sort_index(inplace=True)
    
    return df



In [ ]:
#Use of the load_data function 
path_to_csv = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\Poloniex_BTCUSDT_1h.csv"
df_prices = load_data(path_to_csv)
    
print("DataFrame shape:", df_prices.shape)
print(df_prices.head())
print(df_prices.tail())

series models:

Date/Time : Usually kept as an index.

Open, High, Low, Close (OHLC) : Core price features for any trading or forecasting model.

Volume : Overall volume is often a good indicator of market interest and liquidity.

Trade Count (Optional) : Helps distinguish whether volume came from many small trades or fewer large trades.


In [16]:
# This code Keep only the columns that are most useful for ur dataframe

def keep_important_columns(df):

    # Define the columns you want to keep
    columns_to_keep = [
        "open",
        "high",
        "low",
        "close",
        "Volume BTC",   
        "tradeCount"   
    ]
    
    # Intersect with what actually exists in your DataFrame to avoid KeyErrors
    existing_cols = [col for col in columns_to_keep if col in df.columns]
    
    # Create a reduced DataFrame with only these columns
    df_reduced = df[existing_cols].copy()
    return df_reduced


In [ ]:
# use of keep_important_columns
df_prices = keep_important_columns(df_prices)
print(df_prices)

In [ ]:
# Save the DataFrame to CSV
#df_prices.to_csv('modified_btc_data.csv')


Below is a data-quality checker , It identifies common issues in a crypto price DataFrame, such as:

Missing Values: Looks for NaN/None in columns.

Duplicate Rows: Checks whether any exact duplicates exist.

Negative or Zero Price/Volume: Flags rows where prices or volumes are invalid.

Out-of-Order Date Index: Ensures that your date index is strictly increasing (important for time-series).

Unexpected Data Types: Verifies that “open”, “high”, “low”, “close”, and “volume” columns are numeric.

In [20]:
# Data Quality Checker

import numpy as np

def check_data_problems(df):

    problems_summary = {}

    # 1) Check for missing values
    missing = df.isnull().sum()
    has_missing = missing.any()
    if has_missing:
        print(">> MISSING VALUES found per column:")
        print(missing[missing > 0])
        problems_summary['missing_values'] = missing[missing > 0].to_dict()
    else:
        print("No missing values detected.")
        problems_summary['missing_values'] = {}
    
    # 2) Check for duplicate rows
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        print(f">> DUPLICATE ROWS found: {duplicate_count}")
        problems_summary['duplicate_rows'] = duplicate_count
    else:
        print("No duplicate rows found.")
        problems_summary['duplicate_rows'] = 0
    
    # 3) Negative or zero price/volume checks
    # Adjust these columns to match your DataFrame (e.g., 'Volume BTC', 'Volume USDT', etc.)
    price_cols = [col for col in ['open','high','low','close'] if col in df.columns]
    volume_cols = [col for col in ['Volume BTC','Volume USDT','Volume USD','volume'] if col in df.columns]

    invalid_prices = {}
    for col in price_cols:
        # Count how many rows have a non-positive price
        non_pos = (df[col] <= 0).sum()
        if non_pos > 0:
            invalid_prices[col] = non_pos
    
    invalid_volumes = {}
    for col in volume_cols:
        non_pos = (df[col] <= 0).sum()
        if non_pos > 0:
            invalid_volumes[col] = non_pos

    if invalid_prices:
        print(">> INVALID (≤0) PRICE VALUES found:")
        for c, count in invalid_prices.items():
            print(f"   Column '{c}': {count} rows")
        problems_summary['invalid_prices'] = invalid_prices
    else:
        print("No invalid (zero/negative) price values found.")
        problems_summary['invalid_prices'] = {}
    
    if invalid_volumes:
        print(">> INVALID (≤0) VOLUME VALUES found:")
        for c, count in invalid_volumes.items():
            print(f"   Column '{c}': {count} rows")
        problems_summary['invalid_volumes'] = invalid_volumes
    else:
        print("No invalid (zero/negative) volume values found.")
        problems_summary['invalid_volumes'] = {}

    # 4) Out-of-order date index check (only if index is datetime-like)
    if isinstance(df.index, pd.DatetimeIndex):
        # Check if the index is strictly increasing
        is_increasing = df.index.is_monotonic_increasing
        if not is_increasing:
            print(">> The date index is NOT strictly increasing. Some timestamps may be out of order.")
            problems_summary['date_order'] = "Not strictly increasing"
        else:
            print("Date index is in ascending order (strictly increasing).")
            problems_summary['date_order'] = "Ascending"
    else:
        print("Index is not a DatetimeIndex (skipping date-order check).")
        problems_summary['date_order'] = None

    # 5) Data-type checks for numeric columns
    numeric_checks = {}
    for col in price_cols + volume_cols:
        if col in df.columns:
            if not pd.api.types.is_numeric_dtype(df[col]):
                numeric_checks[col] = "Non-numeric type"
    if numeric_checks:
        print(">> NON-NUMERIC COLUMNS found (expected numeric):")
        for c, msg in numeric_checks.items():
            print(f"   Column '{c}' => {msg}")
    else:
        print("All price/volume columns have numeric types.")
    problems_summary['non_numeric_columns'] = numeric_checks

    print("\n=== DATA QUALITY CHECK COMPLETE ===\n")
    return problems_summary


In [ ]:
# 3. Run the data-quality checker 
problems_report = check_data_problems(df_prices)

# 4. If you want to do something programmatic with the results:
print("Problems Summary (as dict):")
print(problems_report)

'''problems found :
No missing values detected.
>> DUPLICATE ROWS found: 3357
No invalid (zero/negative) price values found.
>> INVALID (≤0) VOLUME VALUES found:
   Column 'Volume BTC': 4458 rows
Date index is in ascending order (strictly increasing).
All price/volume columns have numeric types.

=== DATA QUALITY CHECK COMPLETE ===

Problems Summary (as dict):
{'missing_values': {}, 'duplicate_rows': 3357, 'invalid_prices': {}, 'invalid_volumes': {'Volume BTC': 4458}, 'date_order': 'Ascending', 'non_numeric_columns': {}}
'''

In [ ]:
#this code reads the dataframe from the saved csv
df = pd.read_csv(
    r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\modified_btc_data.csv",
    index_col=0,
    parse_dates=True
)
print(df)


### Deal with Duplicate Rows

In [23]:
# This will return every row that has a duplicate somewhere else
duplicates_all = df[df.duplicated(keep=False)]

print(f"Total rows considered duplicates (including the first occurrence): {len(duplicates_all)}")
print(duplicates_all)


Total rows considered duplicates (including the first occurrence): 0
Empty DataFrame
Columns: [unix, date, symbol, open, high, low, close, Volume BTC, Volume USD]
Index: []


In [24]:
#Count duplicates by year and month
# Ensure index is datetime (just in case)
duplicates_all.index = pd.to_datetime(duplicates_all.index)

# Create year and month columns
duplicates_all['year'] = duplicates_all.index.year
duplicates_all['month'] = duplicates_all.index.month

# Group by year and month, then count rows
duplicate_counts = duplicates_all.groupby(['year', 'month']).size().reset_index(name='count')

# Display
print(duplicate_counts)



Empty DataFrame
Columns: [year, month, count]
Index: []


In [ ]:
#this code compares bettwen the 3 datasets
import pandas as pd

# -------------------------------------------------------------------
# 1) Set your CSV file paths here, named for each exchange
# -------------------------------------------------------------------
poloniex_csv = r"C:\Users\ADMIN\Downloads\Poloniex_BTCUSDT_1h.csv"
bitstamp_csv = r"C:\Users\ADMIN\Downloads\Bitstamp_BTCUSD_1h (1).csv"
gemini_csv   = r"C:\Users\ADMIN\Downloads\2025-2015 Gemini_BTCUSD_1h.csv"

# -------------------------------------------------------------------
# 2) Read each dataset into a DataFrame
# -------------------------------------------------------------------
df_poloniex = pd.read_csv(
    poloniex_csv,
    skiprows=1,
    header=0,
    sep=',',
    parse_dates=['date'],
    index_col='date'
)

df_bitstamp = pd.read_csv(
    bitstamp_csv,
    skiprows=1,
    header=0,
    sep=',',
    parse_dates=['date'],
    index_col='date'
)

df_gemini = pd.read_csv(
    gemini_csv,
    skiprows=1,
    header=0,
    sep=',',
    parse_dates=['date'],
    index_col='date'
)

# -------------------------------------------------------------------
# 3) Align the DataFrames to the common hourly timestamps
#    (only keep rows where all three have data)
# -------------------------------------------------------------------
common_index = df_poloniex.index.intersection(df_bitstamp.index).intersection(df_gemini.index)
df_poloniex = df_poloniex.loc[common_index]
df_bitstamp = df_bitstamp.loc[common_index]
df_gemini   = df_gemini.loc[common_index]

# -------------------------------------------------------------------
# 4) Combine into one comparison table
#    Renaming columns to reflect each exchange name clearly
# -------------------------------------------------------------------
comparison_df = pd.DataFrame(index=common_index)
comparison_df['Poloniex_Close'] = df_poloniex['close']
comparison_df['Bitstamp_Close'] = df_bitstamp['close']
comparison_df['Gemini_Close']   = df_gemini['close']

# -------------------------------------------------------------------
# 5) Calculate the price differences
# -------------------------------------------------------------------
comparison_df['Poloniex_minus_Bitstamp'] = comparison_df['Poloniex_Close'] - comparison_df['Bitstamp_Close']
comparison_df['Bitstamp_minus_Gemini']   = comparison_df['Bitstamp_Close'] - comparison_df['Gemini_Close']
comparison_df['Poloniex_minus_Gemini']   = comparison_df['Poloniex_Close'] - comparison_df['Gemini_Close']

# -------------------------------------------------------------------
# 6) Display the final comparison DataFrame (first 100 rows)
# -------------------------------------------------------------------
print("\n=== Comparison of Poloniex, Bitstamp, and Gemini (Hourly) ===")
print(comparison_df)

# Optionally, you can save the comparison to a CSV file:
# comparison_df.to_csv('comparison_output.csv')


In [26]:
# Save the DataFrame to CSV
#comparison_df.to_csv('comparison_csv')

In [27]:
comparison_df[['Poloniex_minus_Bitstamp', 'Bitstamp_minus_Gemini', 'Poloniex_minus_Gemini']].mean()


#Poloniex_minus_Bitstamp    -9.114675
#Bitstamp_minus_Gemini      -1.109679
#Poloniex_minus_Gemini     -10.224354



Poloniex_minus_Bitstamp    -9.114675
Bitstamp_minus_Gemini      -1.109679
Poloniex_minus_Gemini     -10.224354
dtype: float64

Poloniex_minus_Bitstamp    -9.114675
Bitstamp_minus_Gemini      -1.109679
Poloniex_minus_Gemini     -10.224354
dtype: float64

In [28]:
import numpy as np
import pandas as pd

pairs = [
    ('Poloniex_Close', 'Bitstamp_Close'),
    ('Bitstamp_Close', 'Gemini_Close'),
    ('Poloniex_Close', 'Gemini_Close')
]

for colA, colB in pairs:
    diffs = comparison_df[colA] - comparison_df[colB]
    
    mae  = np.mean(np.abs(diffs))
    rmse = np.sqrt(np.mean(diffs**2))
    corr = comparison_df[colA].corr(comparison_df[colB])
    
    print(f"--- {colA} vs {colB} ---")
    print(f"  MAE : {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  Corr: {corr:.4f}")
    print()


--- Poloniex_Close vs Bitstamp_Close ---
  MAE : 31.4501
  RMSE: 78.8900
  Corr: 1.0000

--- Bitstamp_Close vs Gemini_Close ---
  MAE : 10.5856
  RMSE: 31.9917
  Corr: 1.0000

--- Poloniex_Close vs Gemini_Close ---
  MAE : 33.4455
  RMSE: 84.3110
  Corr: 1.0000



--- Poloniex_Close vs Bitstamp_Close ---
  MAE : 31.4501
  RMSE: 78.8900
  Corr: 1.0000

--- Bitstamp_Close vs Gemini_Close ---
  MAE : 10.5856
  RMSE: 31.9917
  Corr: 1.0000

--- Poloniex_Close vs Gemini_Close ---
  MAE : 33.4455
  RMSE: 84.3110
  Corr: 1.0000
